# 1) Import des librairies et initialisation du Notebook

In [1]:
from lxml import html
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow import keras 
from sklearn import metrics
import sklearn.metrics as metrics
import xgboost as xgb
from sklearn.datasets import dump_svmlight_file
import catboost as cb
from sklearn.metrics import accuracy_score,log_loss
import numpy as np
from sklearn.metrics import precision_score
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import catboost as cb
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

plt.style.use('ggplot')
pd.set_option('display.max_columns',500)

ImportError: No module named 'catboost'

# 2) Scraping des données de train et de test 

### a) Récupération des liens des données

In [ ]:
base_link="https://archive.ics.uci.edu/ml"

html=requests.get(base_link+"/datasets/Poker+Hand").text
soup=BeautifulSoup(html,"lxml")
table=soup.find("body") 
name_part=soup.find("td",{"valign":"top"})

for link in table.find_all("a"):
    if link.get_text()=='Data Folder':
        Data_folder_link=link.get('href')

Data_folder_link=Data_folder_link.replace(".","",2)


html=requests.get(base_link+Data_folder_link ).text
soup=BeautifulSoup(html,"lxml")
table=soup.find("body") 

for link in table.find_all("a"):
    
    if link.get_text()=='poker-hand-testing.data':
        test_link=link.get('href')
    if link.get_text()=='poker-hand-training-true.data':
        train_link=link.get('href')

print(base_link+Data_folder_link+test_link)
print(base_link+Data_folder_link+train_link)

### Création du Datframe test

In [ ]:
test_path = base_link+Data_folder_link+test_link
test = pd.read_csv(test_path, encoding="latin1")
test.head()

### c) Import du Datframe test

In [ ]:
train_path = base_link+Data_folder_link+train_link
train = pd.read_csv(train_path, encoding="latin1")
train.head()

In [ ]:
#Dimension du jeu de train
train.shape

In [ ]:
#Dimension du jeu de test
test.shape

In [ ]:
#Renomage des colonnes des jeux de train et de test
test.columns=train.columns = ['Suit_of_card_1','Rank_of_card_1',
                              'Suit_of_card_2','Rank_of_card_2',
                              'Suit_of_card_3','Rank_of_card_3',
                              'Suit_of_card_4','Rank_of_card_4',
                              'Suit_of_card_5','Rank_of_card_5',
                              'Poker_Hand']


In [ ]:
#Ajout d'une colonne Datasplit Dans les jeux de train et de test
train['datasplit']="train"
test['datasplit']="test"

In [ ]:
train.head()
test.head()

In [ ]:
result = pd.concat([train,test])

In [ ]:
result[25000:]

In [ ]:
result = result.reset_index(drop=True)

In [ ]:
unique_list=pd.unique(result['Poker_Hand'])

# 2) Visualisation de données

### a) Visualisation de la matrice de corrélation des colonnes

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = result.corr()
sn.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sn.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

### b) Visualisation de la répartition des classes de la colonne Poker_Hand

In [ ]:
x=len(unique_list)
datadic={}
for i,obs in enumerate(unique_list):
    datadic[obs] = len(result.loc[(result['Poker_Hand'] == obs)] )
    

In [ ]:
datadic

In [ ]:
poker_hand_dic={0: "Nothing in hand; not a recognized poker hand",
                1: "One pair; one pair of equal ranks within five cards",
                2: "Two pairs; two pairs of equal ranks within five cards",
                3: "Three of a kind; three equal ranks within five cards",
                4: "Straight; five cards, sequentially ranked with no gaps",
                5: "Flush; five cards with the same suit",
                6: "Full house; pair + different rank three of a kind",
                7: "Four of a kind; four equal ranks within five cards",
                8: "Straight flush; straight + flush",
                9: "Royal flush; {Ace, King, Queen, Jack, Ten} + flush" }

###### Graphique de la répartition des classes de la colonne Poker_Hand

In [ ]:
mpl.rcParams['figure.figsize'] = (15,5)

plt.bar(range(len(datadic)), datadic.values(), align='center')
plt.xticks(range(len(datadic)), list(poker_hand_dic.values()),rotation=90)

for a,b in zip(datadic.keys(), datadic.values()): 
     plt.text(a, b, str(b)     
                +" : "+str(round(b/sum(datadic.values())*100))+" %)",      
              horizontalalignment='center',color='blue')

plt.show()

In [ ]:
train=result.loc[(result['datasplit'] == "train")]
train = train.reset_index(drop=True)
train.pop('datasplit')
test=result.loc[(result['datasplit'] == "test")]
test = test.reset_index(drop=True)
test.pop('datasplit')

In [ ]:
test.head()

# 3)Modélisation

### a) Préprocessing

In [ ]:
X_train = train.iloc[:, 0:10].values  
y_train = train.iloc[:, 10].values 
X_test = test.iloc[:, 0:10].values  
y_test = test.iloc[:, 10].values 

In [ ]:
sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test) 

### b) Modèle Random Forest

In [ ]:
regressor = RandomForestRegressor(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  


In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))  

### c) Modèle CatBoost

In [ ]:
#Catboost
#Random seeds are fixed at 1234 to make scores reproducible
def CVandTest(X_train,y_train,X_test,y_test):
    #80/20 train test split cross-validation
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

    cat = cb.CatBoostClassifier(
        loss_function='MultiClassOneVsAll',
        random_seed = 1234
    )
    cat_model1 = cat.fit(X_train,y_train,verbose=False) 
    
    cat_predtrain=cat_model1.predict(X_train, prediction_type='Class')
    
    cat_predA = cat_model1.predict(X_test, prediction_type='Class')
    cat_predLL = cat_model1.predict(X_test, prediction_type='Probability')

    print("CV accuracy: {}".format(accuracy_score(y_test,cat_predA)))
    print("CV logloss: {}".format(log_loss(y_test,cat_predLL)))
    
    #Training with all X,y data, testing with Xte,yte
    cat_model2 = cat.fit(X_test,y_test,verbose=False)

    cat_predAt = cat_model2.predict(X_test, prediction_type='Class')
    cat_predLLt = cat_model2.predict(X_test, prediction_type='Probability')
    

    print("Test accuracy: {}".format(accuracy_score(y_test,cat_predAt)))
    print("Test logloss: {}".format(log_loss(y_test,cat_predLLt)))
    
    return (cat_predtrain,cat_predA, cat_predLL, cat_predAt, cat_predLLt,cat_model1,cat_model2)

In [ ]:
(cat_predtrain,cat_predA, cat_predLL, cat_predAt, cat_predLLt,cat_model1,cat_model2) = CVandTest(X_train,y_train,X_test,y_test)

In [ ]:
confusion_matrix(cat_predAt,y_test)

In [ ]:
array = confusion_matrix(cat_predAt,y_test)
df_cm = pd.DataFrame(array, index = [i for i in ["0","1","2","3","4","5","6","7","8","9"]],
                  columns = [i for i in ["0","1","2","3","4","5","6","7","8","9"]])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

In [ ]:
array = confusion_matrix(cat_predtrain,y_train)
df_cm = pd.DataFrame(array, index = [i for i in ["0","1","2","3","4","5","6","7","8","9"]],
                  columns = [i for i in ["0","1","2","3","4","5","6","7","8","9"]])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)


## d) Modélisation avec du Deep Learning : Keras

In [ ]:
def model(input_shape , output_shape):
    net = keras.Sequential([
        keras.layers.BatchNormalization(),
        keras.layers.Dense(2048 ,input_shape=input_shape , activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(2048 , activation='relu' ),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(output_shape , activation='softmax')
    ])
    
    optimizer = keras.optimizers.Adam(lr=0.001)
    net.compile(optimizer=optimizer , loss='categorical_crossentropy' , metrics=['accuracy'])
    return net

Y_train = keras.utils.to_categorical(y_train)
Y_test = keras.utils.to_categorical(y_test)
net = model((X_train.shape[1] , ) , Y_train.shape[1])
net.fit(X_train , Y_train , epochs=50 , batch_size=128)

In [ ]:
def compute_precision(predictions , y):
    class_diff = predictions - y
    count = 0
    for i in range(class_diff.shape[0]):
        if class_diff[i] == 0:
            count += 1
    return count / class_diff.shape[0]
    

predictions = np.argmax(net.predict(X_train , batch_size=256) , axis=1)
conf_matrix = confusion_matrix(y_train , predictions)

print('\n---Train---\n')
print(conf_matrix)
print('precision : ' , compute_precision(predictions , y_train))
    
predictions = np.argmax(net.predict(X_test , batch_size=256) , axis=1)
conf_matrix = confusion_matrix(y_test , predictions)

print('\n---Test---\n')
print(conf_matrix)
print('precision : ' , compute_precision(predictions , y_test))

In [ ]:
array = compute_precision(predictions , y_train)
df_cm = pd.DataFrame(array, index = [i for i in ["0","1","2","3","4","5","6","7","8","9"]],
                  columns = [i for i in ["0","1","2","3","4","5","6","7","8","9"]])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

In [ ]:
array = compute_precision(predictions , y_test)
df_cm = pd.DataFrame(array, index = [i for i in ["0","1","2","3","4","5","6","7","8","9"]],
                  columns = [i for i in ["0","1","2","3","4","5","6","7","8","9"]])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

## Utilisation de GridSearch avec CatBoost

In [ ]:
params = {
          'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
          'ctr_border_count':[50,5,10,20,100,200]}

In [ ]:
cat = cb.CatBoostClassifier(
        loss_function='MultiClassOneVsAll',
        random_seed = 1234, task_type = "GPU"
    )

In [ ]:
cat = cb.CatBoostClassifier(
        loss_function='MultiClassOneVsAll',
        random_seed = 1234)
cat_model = cat.fit(X_train,y_train,verbose=False) 

In [ ]:
from sklearn import grid_search
         
grid= grid_search.GridSearchCV(cat, params, n_jobs=-1)


In [ ]:
testd=cat.fit(X_train, y_train)

In [ ]:
mod=grid.fit(X_train, y_train)

In [ ]:
print (grid.best_score_, grid.best_estimator_)

In [ ]:
print(grid.best_estimator_.get_params())

In [ ]:
def catboost_param_tune(params,train,train_label,cat_dims=None,n_splits=3):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(ps.grid_search(['border_count']),
                      ps.grid_search(['ctr_border_count']),
                      ps.grid_search(['l2_leaf_reg']),
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['depth'])):
        res = crossvaltest(prms,train_set,train_label,cat_dims,n_splits)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        print(res,prms,s'best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()

bestparams = catboost_param_tune(params,train_set,train_label,cat_dims)

In [ ]:
cat_model1 = cat.fit(X_train,y_train,verbose=False) 